In [28]:
!nvidia-smi

Thu Aug 13 19:32:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |    905MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [11]:
!ls '/content/drive/My Drive'
!ls '/content/drive/My Drive/pytorch'

pytorch
mnist_torch.ipynb


In [1]:
!pip install torch
!pip install torchvision
!pip install torchtext
!pip install torchvision

In [5]:
import torch 
from torch import nn,optim
from torchtext import data,datasets
from torch.nn import functional as F

In [6]:
from torchvision import datasets,transforms

In [7]:
#超参数
batch_size = 200
epochs = 10
learning_rate = 0.001

In [8]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])

In [12]:
train_dataset = datasets.MNIST('minist_data',train=True,transform=transform,download=True)
train_dataset = datasets.MNIST('minist_data',train=True,transform=transform,download=True)


Extracting minist_data/MNIST/raw/train-images-idx3-ubyte.gz to minist_data/MNIST/raw


Extracting minist_data/MNIST/raw/train-labels-idx1-ubyte.gz to minist_data/MNIST/raw


Extracting minist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to minist_data/MNIST/raw


Extracting minist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to minist_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=False)

In [23]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.model = nn.Sequential(
        nn.Linear(784,200),
        nn.ReLU(inplace = True),
        nn.Linear(200,200),
        nn.ReLU(inplace = True),
        nn.Linear(200,10)
        )
    def forward(self,x):
        x = self.model(x)
        return x

In [14]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         # xw + b
#         self.fc1 = nn.Linear(28*28,256)
#         self.fc2 = nn.Linear(256,64)
#         self.fc3 = nn.Linear(64,32)
#         self.fc4 = nn.Linear(32,10)

#     def forward(self, x):
#         # x ; [b, 1, 28, 28]
#         # h1 = sigmoid(xw + b)
#         x = F.relu(self.fc1(x))
#         # h2 = sigmoid(h1w + b)
#         x = F.relu(self.fc2(x))
#         # h3= sigmoid(h2w + b)
#         x = F.relu(self.fc3(x))
#         # h4 = softmax(h3w3 + b3)
#         # x = F.softmax(self.fc4(x))
#         x = self.fc4(x)
#         return x

In [24]:
torch.cuda.is_available()
import logging
# set cuda
gpu = 0
use_cuda = gpu >= 0 and torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(gpu)
    device = torch.device("cuda", gpu)
else:
    device = torch.device("cpu")
logging.info("Use cuda: %s, gpu id: %d.", use_cuda, gpu)

In [25]:
net = MLP().to(device)
criteon = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(),lr = learning_rate)

In [30]:
for epoch in range(epochs):
  for batch_idx,(data,target) in enumerate(train_loader):
    data = data.view(-1,784)
    data,target = data.to(device),target.to(device)
    logits = net(data)
    loss = criteon(logits,target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch_idx %200 == 0:
      print('epoch={0},batch_idx = {1},loss = {2}'.format(epoch,batch_idx,loss))
  test_loss = 0
  correct = 0
  for data,target in test_loader:
    data=data.view(-1,784)
    data,target = data.to(device),target.to(device)
    logits = net(data)
    test_loss+=criteon(logits,target)
    pred = logits.max(dim=1)[1]
    correct+=pred.eq(target).float().sum().item()
  accuracy = correct/float(len(test_loader.dataset))
  total_loss = test_loss/float(len(test_loader.dataset))
  print('Loss={},Accuracy={}'.format(total_loss,accuracy))
  
  

epoch=0,batch_idx = 0,loss = 0.009552201256155968
epoch=0,batch_idx = 200,loss = 0.03309918940067291
Loss=6.718719669152051e-05,Accuracy=0.9955333333333334
epoch=1,batch_idx = 0,loss = 0.002381640486419201
epoch=1,batch_idx = 200,loss = 0.006306233815848827
Loss=6.73424729029648e-05,Accuracy=0.9949833333333333
epoch=2,batch_idx = 0,loss = 0.007376117166131735
epoch=2,batch_idx = 200,loss = 0.014795148745179176
Loss=3.509549060254358e-05,Accuracy=0.9979166666666667
epoch=3,batch_idx = 0,loss = 0.0011350795393809676
epoch=3,batch_idx = 200,loss = 0.002893279306590557
Loss=4.009907570434734e-05,Accuracy=0.9971333333333333
epoch=4,batch_idx = 0,loss = 0.0008942345739342272
epoch=4,batch_idx = 200,loss = 0.0022701947018504143
Loss=9.731060708872974e-05,Accuracy=0.9935666666666667
epoch=5,batch_idx = 0,loss = 0.004924008157104254
epoch=5,batch_idx = 200,loss = 0.0008081170963123441
Loss=5.410624362411909e-05,Accuracy=0.99635
epoch=6,batch_idx = 0,loss = 0.00031225726706907153
epoch=6,batch_i